# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import pickle
import sys

# Set Directory Paths

In [2]:
# Directory path to load formatted Pathology and Thickness Data
%store -r path_dataDir
%store -r thick_dataDir

# Directory path where Data will be saved to
%store -r thickAtPath_dataDir

# [1] Loading Pathology, Thickness Data, and Variables

## Variables

In [3]:
with open(os.path.join(path_dataDir, 'sn.pkl'), 'rb') as f:
    sn = pickle.load(f)
f.close()

## Pathology Data

In [4]:
with open(os.path.join(path_dataDir, 'pathToAtlasIndex.pkl'), 'rb') as f:
    pathToAtlasIndex = pickle.load(f)
f.close()

## Thickness Data

In [5]:
# Loading HCResults
with open(os.path.join(thick_dataDir, 'HCResults.pkl'), 'rb') as f:
    HCResults = pickle.load(f)
f.close()
    
# Loading PatientTAUResults
with open(os.path.join(thick_dataDir, 'PatientTAUResults.pkl'), 'rb') as f:
    PatientTAUResults = pickle.load(f)
f.close()

# Loading PatientTDPResults
with open(os.path.join(thick_dataDir, 'PatientTDPResults.pkl'), 'rb') as f:
    PatientTDPResults = pickle.load(f)
f.close()

# [2] Matching Thickness Data to Pathology Data

## Number of subject for HC, TAU, TDP Data

In [6]:
n_HC = HCResults['Thickness']['Mean'].shape[0]
n_TAU = PatientTAUResults['Thickness']['Mean'].shape[0]
n_TDP = PatientTDPResults['Thickness']['Mean'].shape[0]

## Define Datatype to save Thickness Values Matching Pathology Regions

In [7]:
# Shape --> N x sn x 2 / N - Number of subjects, sn - Nunber of Path Regions, 2 - {L, R}
HCthicknessAtPath = np.empty((n_HC,sn,2))
TAUthicknessAtPath = np.empty((n_TAU,sn,2))
TDPthicknessAtPath = np.empty((n_TDP,sn,2))

## Define Datatype to save Normalized(=divided by total ICV) Volume Values Matching Pathology Regions

In [8]:
HCnormVolumeAtPath = np.empty((n_HC,sn,2))
TAUnormVolumeAtPath = np.empty((n_TAU,sn,2))
TDPnormVolumeAtPath = np.empty((n_TDP,sn,2))

## Matching Thickness Data to Pathology Data

In [9]:
# HC
for n in range(n_HC):
    for p in range(sn):
        for r in range(2):
            curIdx = pathToAtlasIndex[p][r]
            HCthicknessAtPath[n,p,r] = np.mean(HCResults['Thickness']['Mean'][n,curIdx])
            HCnormVolumeAtPath[n,p,r] = np.mean(HCResults['Volume']['Normalized'][n,curIdx])

# TAU
for n in range(n_TAU):
    for p in range(sn):
        for r in range(2):
            curIdx = pathToAtlasIndex[p][r]
            TAUthicknessAtPath[n,p,r] = np.mean(PatientTAUResults['Thickness']['Mean'][n,curIdx])
            TAUnormVolumeAtPath[n,p,r] = np.mean(PatientTAUResults['Volume']['Normalized'][n,curIdx]) # Normalized Volume

# TDP
for n in range(n_TDP):
    for p in range(sn):
        for r in range(2):
            curIdx = pathToAtlasIndex[p][r]
            TDPthicknessAtPath[n,p,r] = np.mean(PatientTDPResults['Thickness']['Mean'][n,curIdx])
            TDPnormVolumeAtPath[n,p,r] = np.mean(PatientTDPResults['Volume']['Normalized'][n,curIdx])

# Formatting The Data

## Convert HCthicknessAtPath, TAUthicknessAtPath, TDPthicknessAtPath to appropriate shape: N x 20 x 2 --> N x 40

In [10]:
HCthicknessAtPath = np.hstack((HCthicknessAtPath[:, :, 0], HCthicknessAtPath[:, :, 1]))# 54 x 40
TAUthicknessAtPath = np.hstack((TAUthicknessAtPath[:, :, 0], TAUthicknessAtPath[:, :, 1])) # 26 x 40
TDPthicknessAtPath = np.hstack((TDPthicknessAtPath[:, :, 0], TDPthicknessAtPath[:, :, 1])) # 30 x 40

## Convert HCnormVolumeAtPath, TAUnormVolumeAtPath, TDPnormVolumeAtPath to appropriate shape: N x 20 x 2 --> N x 40

In [11]:
HCnormVolumeAtPath = np.hstack((HCnormVolumeAtPath[:, :, 0], HCnormVolumeAtPath[:, :, 1]))# 54 x 40
TAUnormVolumeAtPath = np.hstack((TAUnormVolumeAtPath[:, :, 0], TAUnormVolumeAtPath[:, :, 1])) # 26 x 40
TDPnormVolumeAtPath = np.hstack((TDPnormVolumeAtPath[:, :, 0], TDPnormVolumeAtPath[:, :, 1])) # 30 x 40 

# Save the Dataset and Variables

## Save ThicknessAtPath for HC, TAU, and TDP

In [12]:
# Save HCthicknessAtPath
with open(os.path.join(thickAtPath_dataDir, 'HCthicknessAtPath.pkl'), 'wb') as f:
    pickle.dump(HCthicknessAtPath, f)
f.close()

# Save TAUthicknessAtPath
with open(os.path.join(thickAtPath_dataDir, 'TAUthicknessAtPath.pkl'), 'wb') as f:
    pickle.dump(TAUthicknessAtPath, f)
f.close()

# Save TDPthicknessAtPath
with open(os.path.join(thickAtPath_dataDir, 'TDPthicknessAtPath.pkl'), 'wb') as f:
    pickle.dump(TDPthicknessAtPath, f)
f.close()

## Save NormVolumeAtPath for HC, TAU, and TDP

In [13]:
# Save HCnormVolumeAtPath
with open(os.path.join(thickAtPath_dataDir, 'HCnormVolumeAtPath.pkl'), 'wb') as f:
    pickle.dump(HCnormVolumeAtPath, f)
f.close()

# Save TAUnormVolumeAtPath
with open(os.path.join(thickAtPath_dataDir, 'TAUnormVolumeAtPath.pkl'), 'wb') as f:
    pickle.dump(TAUnormVolumeAtPath, f)
f.close()

# Save TDPnormVolumeAtPath
with open(os.path.join(thickAtPath_dataDir, 'TDPnormVolumeAtPath.pkl'), 'wb') as f:
    pickle.dump(TDPnormVolumeAtPath, f)
f.close()